In [26]:
import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [27]:
ds=pd.read_csv('titanic.csv')
ds.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,NaN,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893.0,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894.0,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895.0,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896.0,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
ds.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,417.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1101.000000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.521782,0.481622,0.841838,14.181209,0.896760,0.981429,55.907576
min,893.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,997.000000,0.000000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1101.000000,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1205.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  417 non-null    float64
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(3), int64(4), object(5)
memory usage: 39.3+ KB


In [5]:
ds.isnull().sum()

PassengerId      1
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
ds.dropna(axis=0, how='any', subset=None, inplace=False)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
12,904.0,1,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S
14,906.0,1,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S
24,916.0,1,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",female,48.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C
26,918.0,1,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C
28,920.0,0,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S
...,...,...,...,...,...,...,...,...,...,...,...,...
404,1296.0,0,1,"Frauenthal, Mr. Isaac Gerald",male,43.0,1,0,17765,27.7208,D40,C
405,1297.0,0,2,"Nourney, Mr. Alfred (Baron von Drachstedt"")""",male,20.0,0,0,SC/PARIS 2166,13.8625,D38,C
407,1299.0,0,1,"Widener, Mr. George Dunton",male,50.0,1,1,113503,211.5000,C80,C
411,1303.0,1,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q


In [10]:
ds.drop(['Age', 'Cabin', 'PassengerId'], axis=1, inplace=True) # Remove columns with null values
ds.head()

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked
0,0,3,"Kelly, Mr. James",male,0,0,330911,7.8292,Q
1,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,1,0,363272,7.0000,S
2,0,2,"Myles, Mr. Thomas Francis",male,0,0,240276,9.6875,Q
3,0,3,"Wirz, Mr. Albert",male,0,0,315154,8.6625,S
4,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,1,1,3101298,12.2875,S


In [11]:
ds = ds.dropna(how='any') 

In [12]:
ds.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [14]:
ds.shape

(417, 9)

In [15]:
y = ds['Survived'] # Target variable             
X= ds.drop(['Survived'], axis=1)

In [18]:
numeric_cols = [cname for cname in ds.columns if ds[cname].dtype in ['int64', 'float64']]
X = ds[numeric_cols].copy()
X.head()

,Survived,Pclass,SibSp,Parch,Fare
0,0,3,0,0,7.8292
1,1,3,1,0,7.0000
2,0,2,0,0,9.6875
3,0,3,0,0,8.6625
4,1,3,1,1,12.2875


In [21]:
kf=KFold(n_splits=5)
print("Data is splitinto following number of folds:")
kf.get_n_splits(X,y)

Data is splitinto following number of folds:


5

In [28]:
clf=DecisionTreeClassifier()
print("Accuracies for each fold of data are:")
for train_index, test_index in kf.split(X,y):
    clf.fit(X.iloc[train_index], y.iloc[train_index])
    pred=clf.predict(X.iloc[test_index,:])
    print(round(accuracy_score(y.iloc[test_index],pred),3))

Accuracies for each fold of data are:
1.0
1.0
1.0
1.0
1.0


In [29]:
cv =cross_val_score(DecisionTreeClassifier(),X,y,cv=5,scoring='accuracy')
print("Accuracies of all the folds after the cross validation are:")
cv

Accuracies of all the folds after the cross validation are:


array([1., 1., 1., 1., 1.])